Kiểm tra tổng thể trường và số lượng data

In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler


def inspect_csv(df):
    print("===== CSV OVERVIEW =====")
    print(f"Rows    : {df.shape[0]}")
    print(f"Columns : {df.shape[1]}")
    print("\n--- Data types ---")
    print(df.dtypes)
    print("\n--- Memory usage ---")
    print(round(df.memory_usage(deep=True).sum() / 1024 ** 2, 2), "MB")


df_raw = pd.read_csv("../../data/merge/Khang/Khang.csv")
inspect_csv(df_raw)


===== CSV OVERVIEW =====
Rows    : 778
Columns : 15

--- Data types ---
id                       object
Title                    object
Type                     object
Price_Raw                object
Price_Billion           float64
Area_m2                 float64
District                 object
City                     object
Address                  object
Bedrooms                  int64
Toilets                   int64
Post_Time                object
Link                     object
Description              object
Price_Per_m2_Million    float64
dtype: object

--- Memory usage ---
2.5 MB


Hàm kiểm tra và thông báo ô thiếu dữ liệu theo tung cột

In [3]:
def report_missing_values(df):
    print("===== MISSING VALUE REPORT =====")
    missing = df.isna().sum()
    percent = (missing / len(df)) * 100
    report = pd.DataFrame({
        "Missing_Count": missing,
        "Percent_Missing (%)": percent.round(2)
    }).sort_values(by="Missing_Count", ascending=False)
    print(report)
    return report


In [4]:
report_missing_values(df_raw)

===== MISSING VALUE REPORT =====
                      Missing_Count  Percent_Missing (%)
Price_Per_m2_Million             12                 1.54
Title                             0                 0.00
id                                0                 0.00
Price_Raw                         0                 0.00
Price_Billion                     0                 0.00
Area_m2                           0                 0.00
Type                              0                 0.00
District                          0                 0.00
City                              0                 0.00
Bedrooms                          0                 0.00
Address                           0                 0.00
Toilets                           0                 0.00
Post_Time                         0                 0.00
Link                              0                 0.00
Description                       0                 0.00


,Missing_Count,Percent_Missing (%)
Price_Per_m2_Million,12,1.54
Title,0,0.00
id,0,0.00
Price_Raw,0,0.00
Price_Billion,0,0.00
Area_m2,0,0.00
Type,0,0.00
District,0,0.00
City,0,0.00
Bedrooms,0,0.00


In [5]:
mask = df_raw['Post_Time'].isna()

mask.sum(), len(df_raw)


(np.int64(0), 778)

In [6]:
df_raw['Post_Time'].dtype


dtype('O')

In [7]:
df_raw['Post_Time'].value_counts().head(10)


Post_Time
27/08/2025    54
01/09/2025    45
02/09/2025    42
10/09/2025    23
28/08/2025    22
15/12/2025    22
11/09/2025    20
03/09/2025    20
25/08/2025    20
23/08/2025    17
Name: count, dtype: int64

Parse post time ra datatype datetime

In [8]:
df_raw['Post_Time'] = pd.to_datetime(
    df_raw['Post_Time'],
    format='%d/%m/%Y',
    errors='coerce'
)


In [9]:
df_raw['Post_Time'].dtype


dtype('<M8[ns]')

In [10]:
df_raw['Post_Time'].isna().sum(), len(df_raw)


(np.int64(0), 778)

In [11]:

# --- CẤU HÌNH TÊN FILE ---
INPUT_FILE = '../../data/merge/Khang/Khang.csv'  # Đặt tên file csv của bạn là data.csv
OUTPUT_FILE = '../../data/merge/Khang/Khangdone.csv'

# --- HÀM XỬ LÝ ---
def extract_width(row):
    # Gộp & Làm sạch sơ bộ
    text = str(row.get('Description', '')) + " " + str(row.get('Title', ''))
    text = text.lower()

    # --- BƯỚC CHUẨN HÓA QUAN TRỌNG ---
    text = text.replace(',', '.')           # Đổi phẩy thành chấm (6,5 -> 6.5)
    text = re.sub(r'[❌*×]', 'x', text)     # Đổi mọi icon nhân thành 'x'
    text = re.sub(r'[–\-_:]', ' ', text)    # Đổi gạch nối dính chữ thành khoảng trắng

    # MẪU 1: Cụm "Chiều rộng... 77m" (Xử lý lỗi typo)
    match = re.search(r'chiều\s*rộng\s*(\d+[.]?\d*)', text)
    if match:
        try:
            val = float(match.group(1))
            if val > 50: val = val / 10 # Fix lỗi 77m -> 7.7m
            if 0 < val < 50: return val
        except: pass

    # MẪU 2: Từ khóa "Ngang" + từ đệm (khủng/đẹp) + Số
    match = re.search(r'(?:ngang|rộng|mt)(?:.{0,15}?)\s+(\d+[.]?\d*)', text)
    if match:
        try:
            val = float(match.group(1))
            if 0 < val < 50: return val
        except: pass

    # MẪU 3: Dạng kích thước A x B (Có chữ m hoặc không)
    # Tìm tất cả các cặp số dạng AxB
    matches = list(re.finditer(r'(\d+[.]?\d*)\s*m?\s*x\s*(\d+[.]?\d*)', text))
    if matches:
        for m in matches:
            try:
                n1 = float(m.group(1))
                n2 = float(m.group(2))
                width = min(n1, n2) # Lấy số nhỏ làm chiều ngang

                # Ưu tiên: Nếu dòng đó có chữ 'm' (vd: 6x13m) thì tin luôn
                if 'm' in m.group(0) and 2 <= width < 50:
                    return width
                # Nếu không có 'm' (vd: 4x16), lưu lại xem xét sau
                if 2 <= width < 50:
                    temp_width = width
            except: continue

        # Nếu đã tìm thấy dạng AxB hợp lý
        if 'temp_width' in locals(): return temp_width

    return None

def check_mattien(row):
    text = str(row['Title']) + " " + str(row['Description'])
    text = text.lower()

    # 1. DANH SÁCH TỪ KHÓA LOẠI TRỪ (Thấy mấy từ này là biết ko phải mặt tiền xịn)
    # Ví dụ: "sát mặt tiền", "cách mặt tiền", "gần mặt tiền", "hẻm xe hơi quay đầu"
    exclude_keywords = ['sát mặt tiền', 'cách mặt tiền', 'gần mặt tiền', 'sau mặt tiền']

    for kw in exclude_keywords:
        if kw in text:
            return 0 # Gặp từ 'sát'/'cách' -> Trả về 0 ngay lập tức (Là hẻm)

    # 2. DANH SÁCH TỪ KHÓA NHẬN DIỆN MẶT TIỀN (Sau khi đã lọc bọn 'sát/gần')
    positive_keywords = ['mặt tiền', 'mtkd', 'mt đường', 'lô góc', '2 mặt tiền']

    for kw in positive_keywords:
        if kw in text:
            return 1 # Có mặt tiền

    return 0 # Không thấy gì -> Mặc định là hẻm

# --- CHẠY CHƯƠNG TRÌNH ---
print("Đang xử lý dữ liệu...")

try:
    # 1. Đọc file với đúng các cột bạn cung cấp
    df = pd.read_csv(INPUT_FILE)

    # 2. Xử lý chiều ngang (Width_m)
    df['Width_m'] = df.apply(extract_width, axis=1)

    # 3. Xử lý Mặt tiền (Is_MatTien)
    df['Is_MatTien'] = df.apply(check_mattien, axis=1)

    # 4. Tính Đơn giá (Price_Per_m2) = (Tỷ * 1000) / m2
    # Chỉ tính khi Diện tích > 0 để tránh lỗi chia cho 0
    df['Price_Per_m2'] = df.apply(
        lambda x: round((x['Price_Billion'] * 1000) / x['Area_m2'], 2)
        if x['Area_m2'] > 0 else 0,
        axis=1
    )

    # 5. Lưu file
    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print(f"Xong! File kết quả là: {OUTPUT_FILE}")
    print("-" * 30)
    # In thử 5 dòng các cột quan trọng để kiểm tra
    print(df[['id', 'Title', 'Width_m', 'Is_MatTien', 'Price_Per_m2']].head().to_string())

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{INPUT_FILE}'. Hãy chắc chắn file nằm cùng thư mục với code.")

Đang xử lý dữ liệu...
Xong! File kết quả là: ../../data/merge/Khang/Khangdone.csv
------------------------------
                                     id                                                                                Title  Width_m  Is_MatTien  Price_Per_m2
0  77d338cb-abbc-43e3-9a99-eb76c22adb3f                               Nhà MT đường số phường Chánh Hưng. Giá 5,28 tỷ bớt lộc     3.00           1        128.78
1  34c726fa-59ca-4aaf-91db-683839fea1f8           Bán nhà mặt tiền 3 tầng 6.6x28m giá 16,99 tỷ Đường Trương Thị Hoa     5.00           0        212.38
2  67f9febf-cb8c-4b74-af77-0018eeced7d9                                      Bán nhà mặt tiền thụt Lê Văn Sỹ, 79m2, chỉ 8 tỷ      NaN           1        101.27
3  ca3d7226-45b1-4fb6-a698-536bc479f744                                Nhà Mặt tiền đường Phong Phú - P11, Giá : 5,6 Tỉ (TL)     2.90           1        127.27
4  d9206372-26a0-4aab-bd05-41807c8162e0  TRUNG TÂM QUẬN 9 – SIÊU PHẨM MẶT TIỀN LÃ XUÂN 

Xuất data mà 1 trong các trường Toilets,Bedrooms,  Price_Billion,Price_Per_m2 ,Area_m2  bị thiếu

In [12]:
import pandas as pd

# --- CẤU HÌNH ---
INPUT_FILE = '../../data/merge/Khang/Khangdone.csv'  # File dữ liệu sạch từ bước trước
OUTPUT_FILE = '../../data/merge/Khang/Khangdone1.csv'  # File chứa các dòng bị lỗi

try:
    print(f"Đang đọc file {INPUT_FILE}...")
    df = pd.read_csv(INPUT_FILE)

    # Danh sách các cột cần kiểm tra
    check_cols = ['Address']

    # Lọc các dòng có ít nhất 1 ô bị Null trong danh sách cột trên
    # how='any' nghĩa là chỉ cần 1 cột bị thiếu là lấy ra
    df_missing = df[df[check_cols].isnull().any(axis=1)]

    # Lưu ra file mới
    df_missing.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print("-" * 40)
    print(f"XỬ LÝ XONG!")
    print(f"Tổng số dòng bị thiếu thông tin: {len(df_missing)} dòng")
    print(f"Đã xuất dữ liệu ra file: {OUTPUT_FILE}")
    print("-" * 40)

    # In thống kê chi tiết lại một lần nữa cho tập dữ liệu này
    print("Chi tiết số lượng thiếu trong file này:")
    print(df_missing[check_cols].isnull().sum())

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{INPUT_FILE}'. Hãy chắc chắn bạn đã chạy code tạo file data_clean_final.csv ở bước trước.")
except KeyError as e:
    print(f"Lỗi: File thiếu cột {e}. Hãy kiểm tra lại tên cột trong file CSV.")

Đang đọc file ../../data/merge/Khang/Khangdone.csv...
----------------------------------------
XỬ LÝ XONG!
Tổng số dòng bị thiếu thông tin: 0 dòng
Đã xuất dữ liệu ra file: ../../data/merge/Khang/Khangdone1.csv
----------------------------------------
Chi tiết số lượng thiếu trong file này:
Address    0
dtype: int64


Dựa kết quả lọc width tìm lại trường Area_m2 bị trống, tính lại số tiền trên mỗi m2

In [13]:
import pandas as pd
import re
import numpy as np

# --- CẤU HÌNH ---
INPUT_FILE = '../../data/merge/Khang/Khangdone.csv'
OUTPUT_FILE = '../../data/merge/Khang/Khangdone2.csv'

# ================= 1. GIỮ NGUYÊN HÀM CỦA BẠN (Đã tối ưu) =================
def extract_width(row):
    text = str(row.get('Description', '')) + " " + str(row.get('Title', ''))
    text = text.lower()
    text = text.replace(',', '.')
    text = re.sub(r'[❌*×]', 'x', text)
    text = re.sub(r'[–\-_:]', ' ', text)

    # MẪU 1: Chiều rộng...
    match = re.search(r'chiều\s*rộng\s*(\d+[.]?\d*)', text)
    if match:
        try:
            val = float(match.group(1))
            if val > 50: val = val / 10
            if 0 < val < 50: return val
        except: pass

    # MẪU 2: Ngang...
    match = re.search(r'(?:ngang|rộng|mt)(?:.{0,15}?)\s+(\d+[.]?\d*)', text)
    if match:
        try:
            val = float(match.group(1))
            if 0 < val < 50: return val
        except: pass

    # MẪU 3: AxB
    matches = list(re.finditer(r'(\d+[.]?\d*)\s*m?\s*x\s*(\d+[.]?\d*)', text))
    if matches:
        for m in matches:
            try:
                n1 = float(m.group(1))
                n2 = float(m.group(2))
                width = min(n1, n2)
                if 'm' in m.group(0) and 2 <= width < 50: return width
                if 2 <= width < 50: temp_width = width
            except: continue
        if 'temp_width' in locals(): return temp_width
    return None

def check_mattien(row):
    text = str(row['Title']) + " " + str(row['Description'])
    text = text.lower()
    exclude_keywords = ['sát mặt tiền', 'cách mặt tiền', 'gần mặt tiền', 'sau mặt tiền']
    for kw in exclude_keywords:
        if kw in text: return 0
    positive_keywords = ['mặt tiền', 'mtkd', 'mt đường', 'lô góc', '2 mặt tiền']
    for kw in positive_keywords:
        if kw in text: return 1
    return 0

# ================= 2. HÀM MỚI: TÌM CHIỀU DÀI & TÍNH DIỆN TÍCH =================
def recover_missing_area(row):
    # Nếu Diện tích đã có dữ liệu hợp lệ (>0) thì giữ nguyên, không cần tính lại
    current_area = row.get('Area_m2', 0)
    if pd.notnull(current_area) and current_area > 0:
        return current_area

    # Nếu không có Chiều Ngang (Width), thì chịu thua, không tính được Area
    width = row.get('Width_m')
    if pd.isnull(width) or width == 0:
        return current_area # Trả về như cũ (Null hoặc 0)

    # --- BẮT ĐẦU ĐI TÌM CHIỀU DÀI (LENGTH) ---
    text = str(row.get('Description', '')) + " " + str(row.get('Title', ''))
    text = text.lower()
    text = text.replace(',', '.')
    text = re.sub(r'[❌*×]', 'x', text)
    text = re.sub(r'[–\-_:]', ' ', text)

    length = None

    # MẪU A: Tìm từ khóa "Dài..." (Ví dụ: Ngang 5 dài 20)
    match_dai = re.search(r'dài\s*(\d+[.]?\d*)', text)
    if match_dai:
        try:
            val = float(match_dai.group(1))
            # Chiều dài thường phải lớn hơn chiều rộng và < 100m (nhà phố)
            if val >= width and val < 150:
                length = val
        except: pass

    # MẪU B: Tìm trong cụm AxB (Ví dụ: 5x20)
    # Logic: Nếu Width khớp với 1 trong 2 số, thì số kia là Length
    if length is None:
        matches = list(re.finditer(r'(\d+[.]?\d*)\s*m?\s*x\s*(\d+[.]?\d*)', text))
        for m in matches:
            try:
                n1 = float(m.group(1))
                n2 = float(m.group(2))
                # Nếu 1 trong 2 số xấp xỉ bằng Width mà mình đã tìm ra
                # (Cho sai số 0.1 phòng trường hợp làm tròn)
                if abs(n1 - width) < 0.1: length = n2
                elif abs(n2 - width) < 0.1: length = n1

                if length: break
            except: continue

    # --- TÍNH TOÁN DIỆN TÍCH ---
    if length and length > 0:
        calculated_area = width * length
        return round(calculated_area, 2)

    return current_area # Không tìm thấy Length thì trả về Null như cũ

# --- CHẠY CHƯƠNG TRÌNH ---
print("Đang xử lý dữ liệu...")
try:
    df = pd.read_csv(INPUT_FILE)

    # Bước 1: Trích xuất Width (như cũ)
    print("- Đang trích xuất chiều ngang...")
    df['Width_m'] = df.apply(extract_width, axis=1)

    # Bước 2: CỨU DỮ LIỆU AREA (Bước mới)
    # Lấp đầy các ô Area bị trống bằng cách tính Width * Length
    print("- Đang tính toán để lấp đầy Area bị thiếu...")
    df['Area_m2'] = df.apply(recover_missing_area, axis=1)

    # Bước 3: Xác định mặt tiền
    df['Is_MatTien'] = df.apply(check_mattien, axis=1)

    # Bước 4: Tính lại đơn giá (Sau khi đã cứu được Area)
    df['Price_Per_m2'] = df.apply(
        lambda x: round((x['Price_Billion'] * 1000) / x['Area_m2'], 2)
        if pd.notnull(x['Area_m2']) and x['Area_m2'] > 0 else 0,
        axis=1
    )

    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print(f"Xong! File kết quả: {OUTPUT_FILE}")
    print("-" * 30)

    # In kiểm tra những dòng mà Area được cứu sống (trước đó là null/0 nhưng giờ có Width*Length)
    # Logic in: Có Width, Có Area, nhưng Price_Raw có thể check sau
    print("Mẫu 5 dòng dữ liệu sau khi tính toán:")
    print(df[['id', 'Width_m', 'Area_m2', 'Price_Per_m2']].head(10).to_string())

except Exception as e:
    print(f"Lỗi: {e}")

Đang xử lý dữ liệu...
- Đang trích xuất chiều ngang...
- Đang tính toán để lấp đầy Area bị thiếu...
Xong! File kết quả: ../../data/merge/Khang/Khangdone2.csv
------------------------------
Mẫu 5 dòng dữ liệu sau khi tính toán:
                                     id  Width_m  Area_m2  Price_Per_m2
0  77d338cb-abbc-43e3-9a99-eb76c22adb3f     3.00     41.0        128.78
1  34c726fa-59ca-4aaf-91db-683839fea1f8     5.00     80.0        212.38
2  67f9febf-cb8c-4b74-af77-0018eeced7d9      NaN     79.0        101.27
3  ca3d7226-45b1-4fb6-a698-536bc479f744     2.90     44.0        127.27
4  d9206372-26a0-4aab-bd05-41807c8162e0     7.48    198.0        121.21
5  701f4eb9-7df4-45aa-898d-2f3b100695a4     4.00     80.0        150.00
6  873fd28b-f0e3-4078-8efa-a0033a1e5720     5.00     80.0        128.75
7  329561bc-8bbb-48c1-88d7-2aa4ca485235     6.00    120.0        154.17
8  f4aa3194-1899-42de-a1a4-5c9220e430dc     5.00     90.0        214.44
9  4b0a98c1-3b68-48c8-8799-582fb7774482     5.34     

Trích xuất bổ sung thêm tường Floors cải tiến code trên

In [14]:
import pandas as pd
import re
import numpy as np

# --- CẤU HÌNH ---
# Hãy đảm bảo tên file INPUT đúng với file bạn đang có
INPUT_FILE = '../../data/merge/Khang/Khangdone2.csv'
OUTPUT_FILE = '../../data/merge/Khang/Khangdone3.csv'

# ================= 1. HÀM TRÍCH XUẤT CHIỀU NGANG (WIDTH) =================
def extract_width(row):
    text = str(row.get('Description', '')) + " " + str(row.get('Title', ''))
    text = text.lower()
    text = text.replace(',', '.')
    text = re.sub(r'[❌*×]', 'x', text)
    text = re.sub(r'[–\-_:]', ' ', text)

    # MẪU 1: Chiều rộng...
    match = re.search(r'chiều\s*rộng\s*(\d+[.]?\d*)', text)
    if match:
        try:
            val = float(match.group(1))
            if val > 50: val = val / 10
            if 0 < val < 50: return val
        except: pass

    # MẪU 2: Ngang...
    match = re.search(r'(?:ngang|rộng|mt)(?:.{0,15}?)\s+(\d+[.]?\d*)', text)
    if match:
        try:
            val = float(match.group(1))
            if 0 < val < 50: return val
        except: pass

    # MẪU 3: AxB
    matches = list(re.finditer(r'(\d+[.]?\d*)\s*m?\s*x\s*(\d+[.]?\d*)', text))
    if matches:
        for m in matches:
            try:
                n1 = float(m.group(1))
                n2 = float(m.group(2))
                width = min(n1, n2)
                if 'm' in m.group(0) and 2 <= width < 50: return width
                if 2 <= width < 50: temp_width = width
            except: continue
        if 'temp_width' in locals(): return temp_width
    return None

# ================= 2. HÀM XÁC ĐỊNH MẶT TIỀN =================
def check_mattien(row):
    text = str(row['Title']) + " " + str(row['Description'])
    text = text.lower()
    exclude_keywords = ['sát mặt tiền', 'cách mặt tiền', 'gần mặt tiền', 'sau mặt tiền']
    for kw in exclude_keywords:
        if kw in text: return 0
    positive_keywords = ['mặt tiền', 'mtkd', 'mt đường', 'lô góc', '2 mặt tiền']
    for kw in positive_keywords:
        if kw in text: return 1
    return 0

# ================= 3. HÀM TÍNH TOÁN LẠI DIỆN TÍCH =================
def recover_missing_area(row):
    current_area = row.get('Area_m2', 0)
    if pd.notnull(current_area) and current_area > 0:
        return current_area

    width = row.get('Width_m')
    if pd.isnull(width) or width == 0:
        return current_area

    text = str(row.get('Description', '')) + " " + str(row.get('Title', ''))
    text = text.lower()
    text = text.replace(',', '.')
    text = re.sub(r'[❌*×]', 'x', text)
    text = re.sub(r'[–\-_:]', ' ', text)

    length = None
    # MẪU A: Tìm từ khóa "Dài..."
    match_dai = re.search(r'dài\s*(\d+[.]?\d*)', text)
    if match_dai:
        try:
            val = float(match_dai.group(1))
            if val >= width and val < 150:
                length = val
        except: pass

    # MẪU B: Tìm trong cụm AxB
    if length is None:
        matches = list(re.finditer(r'(\d+[.]?\d*)\s*m?\s*x\s*(\d+[.]?\d*)', text))
        for m in matches:
            try:
                n1 = float(m.group(1))
                n2 = float(m.group(2))
                if abs(n1 - width) < 0.1: length = n2
                elif abs(n2 - width) < 0.1: length = n1
                if length: break
            except: continue

    if length and length > 0:
        return round(width * length, 2)
    return current_area

# ================= 4. HÀM TRÍCH XUẤT SỐ TẦNG (FLOORS) - MỚI! =================
def extract_floors(row):
    text = str(row.get('Description', '')) + " " + str(row.get('Title', ''))
    text = text.lower()
    text = text.replace('tấm', 'tầng') # Chuẩn hóa

    floors = 0
    found_sDatture = False

    # 4.1 Chiến thuật cộng dồn: Trệt + Lầu + Lửng + Sân thượng
    matches_lau = re.findall(r'(\d+)\s*lầu', text)
    if matches_lau:
        num_lau = max([float(x) for x in matches_lau])
        floors = num_lau + 1 # +1 cho tầng trệt
        found_sDatture = True
    elif 'trệt' in text and 'lầu' in text:
        floors = 2 # Mặc định 1 trệt 1 lầu
        found_sDatture = True

    if found_sDatture:
        if 'lửng' in text: floors += 0.5
        if 'sân thượng' in text or 'chuồng cu' in text or 'tum' in text: floors += 0.5
        return floors

    # 4.2 Chiến thuật tìm số tổng: "3 tầng", "4 tấm"
    match_tang = re.search(r'(\d+)\s*tầng', text)
    if match_tang:
        return float(match_tang.group(1))

    # 4.3 Các trường hợp đặc biệt
    if 'cấp 4' in text or 'cap 4' in text: return 1.0
    if 'gác' in text: return 1.5

    return 1.0 # Mặc định bèo nhất là 1 tầng

# ================= CHẠY CHƯƠNG TRÌNH =================
print("Đang xử lý dữ liệu...")
try:
    df = pd.read_csv(INPUT_FILE)

    # Bước 1: Trích xuất Width
    print("- Đang trích xuất chiều ngang (Width)...")
    df['Width_m'] = df.apply(extract_width, axis=1)

    # Bước 2: CỨU DỮ LIỆU AREA
    print("- Đang tính toán để lấp đầy Area bị thiếu...")
    df['Area_m2'] = df.apply(recover_missing_area, axis=1)

    # Bước 3: Xác định mặt tiền
    print("- Đang xác định Mặt tiền (Is_MatTien)...")
    df['Is_MatTien'] = df.apply(check_mattien, axis=1)

    # Bước 4: Trích xuất số tầng (MỚI)
    print("- Đang đọc kết cấu số tầng (Floors)...")
    df['Floors'] = df.apply(extract_floors, axis=1)

    # Bước 5: Tính lại đơn giá
    print("- Đang cập nhật đơn giá (Price_Per_m2)...")
    df['Price_Per_m2'] = df.apply(
        lambda x: round((x['Price_Billion'] * 1000) / x['Area_m2'], 2)
        if pd.notnull(x['Area_m2']) and x['Area_m2'] > 0 else 0,
        axis=1
    )

    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print("-" * 30)
    print(f"✅ XONG! File kết quả đầy đủ: {OUTPUT_FILE}")
    print("-" * 30)

    # In kiểm tra mẫu
    print("Mẫu 5 dòng dữ liệu (Width, Area, Floors, Price/m2):")
    print(df[['id', 'Width_m', 'Area_m2', 'Floors', 'Price_Per_m2']].head(10).to_string())

except Exception as e:
    print(f"❌ Lỗi: {e}")

Đang xử lý dữ liệu...
- Đang trích xuất chiều ngang (Width)...
- Đang tính toán để lấp đầy Area bị thiếu...
- Đang xác định Mặt tiền (Is_MatTien)...
- Đang đọc kết cấu số tầng (Floors)...
- Đang cập nhật đơn giá (Price_Per_m2)...
------------------------------
✅ XONG! File kết quả đầy đủ: ../../data/merge/Khang/Khangdone3.csv
------------------------------
Mẫu 5 dòng dữ liệu (Width, Area, Floors, Price/m2):
                                     id  Width_m  Area_m2  Floors  Price_Per_m2
0  77d338cb-abbc-43e3-9a99-eb76c22adb3f     3.00     41.0     3.0        128.78
1  34c726fa-59ca-4aaf-91db-683839fea1f8     5.00     80.0     3.5        212.38
2  67f9febf-cb8c-4b74-af77-0018eeced7d9      NaN     79.0     1.0        101.27
3  ca3d7226-45b1-4fb6-a698-536bc479f744     2.90     44.0     2.0        127.27
4  d9206372-26a0-4aab-bd05-41807c8162e0     7.48    198.0     2.0        121.21
5  701f4eb9-7df4-45aa-898d-2f3b100695a4     4.00     80.0     3.0        150.00
6  873fd28b-f0e3-4078-8efa-a0

Lọc Thiếu Price_Per_m2, Thiếu District, Giá quá rẻ (< 500 triệu), Width_m, trùng lặp, Điền khuyết thông minh: Tự động tính Median số phòng ngủ/toilet của từng Quận để điền vào chỗ trống của Quận đó.

In [15]:
import pandas as pd
import numpy as np

# --- CẤU HÌNH ---
# Input là file đã tính toán xong ở bước trước
INPUT_FILE = '../../data/merge/Khang/Khangdone3.csv'
OUTPUT_FILE = '../../data/merge/Khang/Khangdone4.csv'

try:
    print("⏳ Đang đọc dữ liệu...")
    df = pd.read_csv(INPUT_FILE)
    n_original = len(df)
    print(f"--> Tổng số dòng ban đầu: {n_original}")

    # ================= BƯỚC 1: THANH LỌC DỮ LIỆU (FILTERING) =================

    # 1.1 Xóa các dòng bị thiếu thông tin CHÍ MẠNG (Critical Missing)
    # Thêm 'Area_m2' vào đây để loại bỏ ngay 49 dòng bị lỗi diện tích
    critical_cols = ['Price_Per_m2', 'District', 'Width_m', 'Area_m2']
    df = df.dropna(subset=critical_cols)

    # Lọc thêm: Đảm bảo đơn giá phải > 0 (tránh lỗi chia cho 0 còn sót)
    df = df[df['Price_Per_m2'] > 0]

    print(f"- Sau khi xóa thiếu Giá/Quận/Chiều ngang/Diện tích: {len(df)}")

    # 1.2 Lọc theo Giá trị (Price Billion)
    # Yêu cầu: Giá quá rẻ (< 500 triệu hay 0.5 tỷ) -> XÓA
    df = df[df['Price_Billion'] >= 0.5]
    print(f"- Sau khi xóa nhà < 500 triệu: {len(df)}")

    # 1.3 Xử lý trùng lặp (Duplicates)
    # Logic: Nếu Tiêu đề, Giá, Diện tích và Quận giống hệt nhau -> Coi là spam -> XÓA
    df = df.drop_duplicates(subset=['Title', 'Price_Billion', 'Area_m2', 'District'])
    print(f"- Sau khi xóa tin Spam trùng lặp: {len(df)}")

    # *LƯU Ý: Các điều kiện Diện tích <10, >500, Giá ảo >800 -> KHÔNG LỌC (Theo yêu cầu)*

    # ================= BƯỚC 2: ĐIỀN KHUYẾT THÔNG MINH (GROUP IMPUTATION) =================
    # Yêu cầu: Dùng Median (Trung vị) của từng QUẬN để điền vào Bedrooms/Toilets bị thiếu

    print("\n⏳ Đang tính toán Median theo từng Quận để điền khuyết...")

    # Hàm điền median theo nhóm
    def fill_na_with_district_median(df, target_col, group_col='District'):
        # Tính median cho từng nhóm (Quận)
        median_series = df.groupby(group_col)[target_col].transform('median')

        # Điền các ô NaN bằng giá trị median vừa tính
        df[target_col] = df[target_col].fillna(median_series)

        # "Chữa cháy": Nếu cả Quận đó không có dữ liệu nào (hiếm), dùng Median toàn thành phố
        global_median = df[target_col].median()
        df[target_col] = df[target_col].fillna(global_median)

        # Làm tròn số (vì phòng ngủ/toilet phải là số nguyên)
        df[target_col] = df[target_col].round()
        return df

    # Thực hiện điền cho Bedrooms & Toilets
    df = fill_na_with_district_median(df, 'Bedrooms')
    df = fill_na_with_district_median(df, 'Toilets')

        # ================= BƯỚC 3: KIỂM TRA CUỐI CÙNG & LƯU FILE =================

    print("\n⏳ Đang lưu file kết quả...")

    # Lưu file CSV
    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print("-" * 50)
    print("✅ HOÀN TẤT XỬ LÝ DỮ LIỆU")
    print(f"📁 File kết quả: {OUTPUT_FILE}")
    print(f"📊 Số lượng dòng còn lại: {len(df)} "
          f"(Giữ lại {round(len(df)/n_original*100, 1)}% dữ liệu gốc)")
    print("-" * 50)

    # ================= DEMO PHỤC VỤ BÁO CÁO =================
    print("\n[Demo Báo Cáo] Kiểm tra dữ liệu Quận Gò Vấp sau khi điền Median:")

    sample = df[df['District'].str.contains('Gò Vấp', na=False, case=False)]
    if not sample.empty:
        print(sample[['District', 'Bedrooms', 'Toilets']].head(5).to_string())
    else:
        print("(Không có dữ liệu Gò Vấp – in 5 dòng bất kỳ)")
        print(df[['District', 'Bedrooms', 'Toilets']].head(5).to_string())

except Exception as e:
    print("❌ CÓ LỖI XẢY RA TRONG QUÁ TRÌNH XỬ LÝ!")
    print(f"👉 Chi tiết lỗi: {e}")


⏳ Đang đọc dữ liệu...
--> Tổng số dòng ban đầu: 778
- Sau khi xóa thiếu Giá/Quận/Chiều ngang/Diện tích: 665
- Sau khi xóa nhà < 500 triệu: 662
- Sau khi xóa tin Spam trùng lặp: 557

⏳ Đang tính toán Median theo từng Quận để điền khuyết...

⏳ Đang lưu file kết quả...
--------------------------------------------------
✅ HOÀN TẤT XỬ LÝ DỮ LIỆU
📁 File kết quả: ../../data/merge/Khang/Khangdone4.csv
📊 Số lượng dòng còn lại: 557 (Giữ lại 71.6% dữ liệu gốc)
--------------------------------------------------

[Demo Báo Cáo] Kiểm tra dữ liệu Quận Gò Vấp sau khi điền Median:
        District  Bedrooms  Toilets
8    Quận Gò Vấp         4        4
90   Quận Gò Vấp         3        2
132  Quận Gò Vấp         3        4
140  Quận Gò Vấp         2        0
177  Quận Gò Vấp         4        2


Từ title + description => phường(ward)

In [16]:
import pandas as pd
import re

# --- CẤU HÌNH ---
INPUT_FILE = '../../data/merge/Khang/Khangdone4.csv'
OUTPUT_FILE = '../../data/merge/Khang/Khangdone5.csv'

# 1. TỪ ĐIỂN PHƯỜNG/XÃ (Đã tách riêng Q2, Q9, Thủ Đức để chuẩn xác hơn)
hcm_wards_advanced = {
    'Quận 1': ['Bến Nghé', 'Bến Thành', 'Cô Giang', 'Cầu Kho', 'Cầu Ông Lãnh', 'Đa Kao', 'Nguyễn Cư Trinh', 'Nguyễn Thái Bình', 'Phạm Ngũ Lão', 'Tân Định'],
    'Quận 3': ['1', '2', '3', '4', '5', '9', '10', '11', '12', '14', 'Võ Thị Sáu', '6', '7', '8'],
    'Quận 4': ['1', '2', '3', '4', '6', '8', '9', '10', '13', '14', '15', '16', '18', '12'],
    'Quận 5': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'],
    'Quận 6': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14'],
    'Quận 7': ['Bình Thuận', 'Phú Mỹ', 'Phú Thuận', 'Tân Hưng', 'Tân Kiểng', 'Tân Phong', 'Tân Phú', 'Tân Quy', 'Tân Thuận Đông', 'Tân Thuận Tây'],
    'Quận 8': ['1', '2', '3', '4', '5', '6', '8', '7', '9', '10', '11', '12', '13', '14', '15', '16'],
    'Quận 10': ['1', '2', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '3'],
    'Quận 11': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'],
    'Quận 12': ['An Phú Đông', 'Đông Hưng Thuận', 'Hiệp Thành', 'Tân Chánh Hiệp', 'Tân Hưng Thuận', 'Tân Thới Hiệp', 'Tân Thới Nhất', 'Thạnh Lộc', 'Thạnh Xuân', 'Thới An', 'Trung Mỹ Tây'],
    'Bình Thạnh': ['1', '2', '3', '5', '6', '7', '11', '12', '13', '14', '15', '17', '19', '21', '22', '24', '25', '26', '27', '28'],
    'Gò Vấp': ['1', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17'],
    'Phú Nhuận': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11', '13', '15', '17'],
    'Tân Bình': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'],
    'Tân Phú': ['Hiệp Tân', 'Hòa Thạnh', 'Phú Thọ Hòa', 'Phú Thạnh', 'Phú Trung', 'Sơn Kỳ', 'Tân Quý', 'Tân Sơn Nhì', 'Tân Thành', 'Tân Thới Hòa', 'Tây Thạnh'],
    'Bình Tân': ['An Lạc', 'An Lạc A', 'Bình Hưng Hòa', 'Bình Hưng Hòa A', 'Bình Hưng Hòa B', 'Bình Trị Đông', 'Bình Trị Đông A', 'Bình Trị Đông B', 'Tân Tạo', 'Tân Tạo A'],

    # Khu vực TP Thủ Đức (Tách ra để chính xác)
    'Thủ Đức': ['Bình Chiểu', 'Bình Thọ', 'Hiệp Bình Chánh', 'Hiệp Bình Phước', 'Linh Chiểu', 'Linh Đông', 'Linh Tây', 'Linh Trung', 'Linh Xuân', 'Tam Bình', 'Tam Phú', 'Trường Thọ'],
    'Quận 2': ['An Khánh', 'An Lợi Đông', 'An Phú', 'Bình An', 'Bình Khánh', 'Bình Trưng Đông', 'Bình Trưng Tây', 'Cát Lái', 'Thạnh Mỹ Lợi', 'Thảo Điền', 'Thủ Thiêm'],
    'Quận 9': ['Hiệp Phú', 'Long Bình', 'Long Phước', 'Long Thạnh Mỹ', 'Long Trường', 'Phú Hữu', 'Phước Bình', 'Phước Long A', 'Phước Long B', 'Tân Phú', 'Tăng Nhơn Phú A', 'Tăng Nhơn Phú B'],

    # Huyện
    'Củ Chi': ['An Nhơn Tây', 'Bình Mỹ', 'Hòa Phú', 'Nhuận Đức', 'Phạm Văn Cội', 'Phú Hòa Đông', 'Phú Mỹ Hưng', 'Phước Hiệp', 'Phước Thạnh', 'Phước Vĩnh An', 'Tân An Hội', 'Tân Phú Trung', 'Tân Thạnh Đông', 'Tân Thạnh Tây', 'Tân Thông Hội', 'Thái Mỹ', 'Trung An', 'Trung Lập Hạ', 'Trung Lập Thượng', 'Củ Chi', 'Thị trấn Củ Chi'],
    'Hóc Môn': ['Bà Điểm', 'Đông Thạnh', 'Nhị Bình', 'Tân Hiệp', 'Tân Thới Nhì', 'Tân Xuân', 'Thới Tam Thôn', 'Trung Chánh', 'Xuân Thới Đông', 'Xuân Thới Sơn', 'Xuân Thới Thượng', 'Hóc Môn', 'Thị trấn Hóc Môn'],
    'Bình Chánh': ['An Phú Tây', 'Bình Chánh', 'Bình Hưng', 'Bình Lợi', 'Đa Phước', 'Hưng Long', 'Lê Minh Xuân', 'Phạm Văn Hai', 'Phong Phú', 'Quy Đức', 'Tân Kiên', 'Tân Nhựt', 'Tân Quý Tây', 'Vĩnh Lộc A', 'Vĩnh Lộc B', 'Tân Túc', 'Thị trấn Tân Túc'],
    'Nhà Bè': ['Hiệp Phước', 'Long Thới', 'Nhơn Đức', 'Phú Xuân', 'Phước Kiển', 'Phước Lộc', 'Nhà Bè', 'Thị trấn Nhà Bè'],
    'Cần Giờ': ['An Thới Đông', 'Bình Khánh', 'Long Hòa', 'Lý Nhơn', 'Tam Thôn Hiệp', 'Thạnh An', 'Cần Thạnh', 'Thị trấn Cần Thạnh']
}

# 2. MAPPING SÁP NHẬP (QUAN TRỌNG: MAP MỚI VỀ CŨ)
# Cấu trúc: ('Quận', 'Tên Mới'): 'Tên Cũ Đại Diện'
merged_ward_mapping = {
    # Gò Vấp: Hạnh Thông = P1 + P3 (Chọn P1)
    ('Gò Vấp', 'Hạnh Thông'): '1',
    ('Gò Vấp', 'Phường Hạnh Thông'): '1',

    # Quận 3: Võ Thị Sáu = P6 + P7 + P8 (Chọn P6)
    ('Quận 3', 'Võ Thị Sáu'): '6',

    # Quận 2 (Thủ Đức): An Khánh mới = Bình An + Bình Khánh (Chọn Bình An)
    ('Quận 2', 'An Khánh'): 'Bình An',
    ('Thủ Đức', 'An Khánh'): 'Bình An',

    # Quận 4: P12 nhập vào P13
    ('Quận 4', '12'): '13',

    # Quận 5: P15 nhập vào P12
    ('Quận 5', '15'): '12',

    # Quận 10: P3 nhập vào P2
    ('Quận 10', '3'): '2',

    # Phú Nhuận: P12 -> P11, P14 -> P13 (Ví dụ)
    ('Phú Nhuận', '12'): '11',
    ('Phú Nhuận', '14'): '13'
}

def normalize_district_key(district_raw):
    if pd.isna(district_raw): return None
    d = str(district_raw).strip()
    if d in ['Q2', 'Q.2', 'Quận 2', 'District 2']: return 'Quận 2'
    if d in ['Q9', 'Q.9', 'Quận 9', 'District 9']: return 'Quận 9'
    if d in ['Thủ Đức', 'Q.Thủ Đức', 'TP Thủ Đức', 'Thành phố Thủ Đức']: return 'Thủ Đức'
    if d in hcm_wards_advanced: return d
    if d.startswith('Q') and d[1:].isdigit(): return f"Quận {d[1:]}"
    if d.startswith('Q.') and d[2:].isdigit(): return f"Quận {d[2:]}"
    return d

def extract_ward(row):
    district_key = normalize_district_key(row['District'])
    if not district_key: return None

    # Gộp text để tìm kiếm
    text_search = (str(row.get('Title', '')) + ' ' + str(row.get('Description', '')) + ' ' + str(row.get('Address', ''))).lower()

    # --- ƯU TIÊN 1: CHECK MAPPING SÁP NHẬP (TÊN MỚI) ---
    for (dist, new_ward), old_ward_target in merged_ward_mapping.items():
        # Kiểm tra đúng quận (hoặc quận thuộc nhóm Thủ Đức)
        is_match_district = (dist == district_key) or \
                            (district_key == 'Thủ Đức' and dist in ['Quận 2', 'Quận 9', 'Thủ Đức'])

        if is_match_district:
            # Nếu tìm thấy tên phường mới (VD: "Hạnh Thông") trong văn bản
            if new_ward.lower() in text_search:
                return old_ward_target # Trả về tên cũ (VD: "1")

    # --- ƯU TIÊN 2: TÌM KIẾM BÌNH THƯỜNG ---
    possible_wards = []
    if district_key in hcm_wards_advanced:
        possible_wards = hcm_wards_advanced[district_key]
    elif district_key in ['Thành phố Thủ Đức', 'TP. Thủ Đức']:
         # Nếu input chỉ ghi chung là TP Thủ Đức, ta tìm trong cả 3 quận cũ
         possible_wards = hcm_wards_advanced['Thủ Đức'] + hcm_wards_advanced['Quận 2'] + hcm_wards_advanced['Quận 9']
    else:
        return None

    # Tìm Phường Số (P1, F.1, Phường 1...)
    numeric_wards = [w for w in possible_wards if w.isdigit()]
    for ward in numeric_wards:
        # Regex thông minh: bắt p, f, phường + số + không có số đằng sau
        pattern = r'(?:phường|p|f)[\.\s]*0?' + ward + r'(?!\d)'
        if re.search(pattern, text_search):
            return ward

    # Tìm Phường Chữ (Tân Định, Đa Kao...)
    named_wards = [w for w in possible_wards if not w.isdigit()]
    named_wards.sort(key=len, reverse=True) # Tìm tên dài trước

    for ward in named_wards:
        if ward.lower() in text_search:
            return ward

    return None

# --- CHẠY CHƯƠNG TRÌNH ---
print("Đang xử lý trích xuất Phường (Advanced Mapping)...")
try:
    df = pd.read_csv(INPUT_FILE)

    df['Ward'] = df.apply(extract_ward, axis=1)

    # Thống kê
    total = len(df)
    found = df['Ward'].notna().sum()
    print(f"Tổng số dòng: {total}")
    print(f"Tìm thấy Phường: {found} ({found/total*100:.2f}%)")
    print("-" * 30)
    print(df[['District', 'Ward']].head(10))

    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
    print(f"Đã lưu file kết quả: {OUTPUT_FILE}")

except Exception as e:
    print(f"Lỗi: {e}")

Đang xử lý trích xuất Phường (Advanced Mapping)...
Tổng số dòng: 557
Tìm thấy Phường: 279 (50.09%)
------------------------------
       District             Ward
0        Quận 8             None
1       Quận 12    Tân Thới Hiệp
2        Quận 8               11
3        Quận 9  Tăng Nhơn Phú A
4        Quận 7          Tân Quy
5  Huyện Nhà Bè             None
6  Quận Thủ Đức             None
7   Quận Gò Vấp             None
8        Quận 7    Tân Thuận Tây
9  Quận Thủ Đức             None
Đã lưu file kết quả: ../../data/merge/Khang/Khangdone5.csv


Lọc chỉ lấy những cột có trường Ward không null


In [18]:
import pandas as pd

# --- CẤU HÌNH ---
INPUT_FILE = '../../data/merge/Khang/Khangdone5.csv'  # File vừa chạy xong ở bước trước
OUTPUT_FILE = '../../data/merge/Khang/Khangdone6.csv'  # File SẠCH SẼ (Chỉ chứa data có Ward)

try:
    print(f"⏳ Đang đọc file '{INPUT_FILE}'...")
    df = pd.read_csv(INPUT_FILE)

    initial_count = len(df)

    # 1. LỌC DỮ LIỆU
    # Giữ lại các dòng mà cột 'Ward' KHÔNG bị rỗng (notna)
    df_valid = df[df['Ward'].notna()]

    # 2. LƯU FILE
    df_valid.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    # 3. BÁO CÁO KẾT QUẢ
    valid_count = len(df_valid)
    dropped_count = initial_count - valid_count

    print("-" * 50)
    print(f"✅ ĐÃ XUẤT THÀNH CÔNG FILE: {OUTPUT_FILE}")
    print("-" * 50)
    print(f"📊 Tổng số dòng ban đầu: {initial_count}")
    print(f"🟢 Số dòng HỢP LỆ (Có Phường): {valid_count} dòng")
    print(f"🔴 Số dòng BỊ LOẠI (Null Phường): {dropped_count} dòng")
    print("-" * 50)

    # In mẫu để kiểm tra
    print("\nMẫu 5 dòng dữ liệu chuẩn đã lọc:")
    print(df_valid[['District', 'Ward', 'Title']].head(5).to_string())

except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file '{INPUT_FILE}'. Bạn hãy chắc chắn đã chạy code trích xuất Phường ở bước trước.")
except KeyError:
    print("❌ Lỗi: File không có cột 'Ward'. Hãy kiểm tra lại.")

⏳ Đang đọc file '../../data/merge/Khang/Khangdone5.csv'...
--------------------------------------------------
✅ ĐÃ XUẤT THÀNH CÔNG FILE: ../../data/merge/Khang/Khangdone6.csv
--------------------------------------------------
📊 Tổng số dòng ban đầu: 557
🟢 Số dòng HỢP LỆ (Có Phường): 279 dòng
🔴 Số dòng BỊ LOẠI (Null Phường): 278 dòng
--------------------------------------------------

Mẫu 5 dòng dữ liệu chuẩn đã lọc:
  District             Ward                                                                                     Title
1  Quận 12    Tân Thới Hiệp                Bán nhà mặt tiền 3 tầng 6.6x28m giá 16,99 tỷ Đường Trương Thị Hoa
2   Quận 8               11                                     Nhà Mặt tiền đường Phong Phú - P11, Giá : 5,6 Tỉ (TL)
3   Quận 9  Tăng Nhơn Phú A       TRUNG TÂM QUẬN 9 – SIÊU PHẨM MẶT TIỀN LÃ XUÂN OAI – 198M² – 2 TẦNG BTCT – GIÁ 24 TỶ
4   Quận 7          Tân Quy        SANG GẤP NHÀ MT ĐƯỜNG SỐ - NGANG 4M - 3 TẦNG - P. TÂN QUY, Q7 - NHỈNH 12 

Lọc ra các trường cần thiết

In [20]:
import pandas as pd

# --- CẤU HÌNH ---
INPUT_FILE = '../../data/merge/Khang/Khangdone6.csv'  # File đã có đủ Phường
OUTPUT_FILE = '../../data/merge/Khang_done.csv'  # File kết quả

try:
    print(f"⏳ Đang đọc file '{INPUT_FILE}'...")
    df = pd.read_csv(INPUT_FILE)

    # 1. XỬ LÝ CỘT "IS HEM" (Tạo từ Is_MatTien)
    # Logic: Nếu Is_MatTien = 0 thì Is Hem = 1
    if 'Is Hem' not in df.columns:
        print("- Đang tạo cột 'Is Hem'...")
        df['Is Hem'] = df['Is_MatTien'].apply(lambda x: 1 if x == 0 else 0)

    # 2. XỬ LÝ CỘT "POST TIME"
    # Trong code cũ tên là 'Post_Time' (có gạch dưới), nếu bạn muốn tên là 'Post Time' (có dấu cách) thì đổi lại
    if 'Post_Time' in df.columns:
        df.rename(columns={'Post_Time': 'Post Time'}, inplace=True)

    # 3. DANH SÁCH 12 CỘT BẠN YÊU CẦU
    target_cols = [
        'Price_Billion',
        'Price_Per_m2',
        'Area_m2',
        'District',
        'Ward',
        'Bedrooms',
        'Is_MatTien',
        'Width_m',
        'Floors',
        'Is Hem',
        'Post Time',
        'Toilets'
    ]

    # 4. THỰC HIỆN LỌC VÀ LƯU FILE
    # Chỉ lấy đúng các cột trong danh sách
    df_final = df[target_cols]

    df_final.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print("-" * 50)
    print(f"✅ ĐÃ XONG! File chỉ chứa đúng 12 cột: {OUTPUT_FILE}")
    print("-" * 50)
    print(df_final.head().to_string())

except KeyError as e:
    print(f"❌ Lỗi: Không tìm thấy cột {e} trong file gốc. Hãy kiểm tra lại tên cột.")
except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file '{INPUT_FILE}'")
except Exception as e:
    print(f"❌ Lỗi: {e}")

⏳ Đang đọc file '../../data/merge/Khang/Khangdone6.csv'...
- Đang tạo cột 'Is Hem'...
--------------------------------------------------
✅ ĐÃ XONG! File chỉ chứa đúng 12 cột: ../../data/merge/Khang_done.csv
--------------------------------------------------
   Price_Billion  Price_Per_m2  Area_m2 District             Ward  Bedrooms  Is_MatTien  Width_m  Floors  Is Hem   Post Time  Toilets
0          16.99        212.38     80.0  Quận 12    Tân Thới Hiệp         5           0     5.00     3.5       1  18/11/2025        5
1           5.60        127.27     44.0   Quận 8               11         2           1     2.90     2.0       0  16/12/2025        3
2          24.00        121.21    198.0   Quận 9  Tăng Nhơn Phú A         4           1     7.48     2.0       0  04/12/2025        4
3          12.00        150.00     80.0   Quận 7          Tân Quy         2           1     4.00     3.0       0  18/11/2025        3
4          11.10        179.03     62.0   Quận 7    Tân Thuận Tây       

In [21]:
import pandas as pd
import numpy as np
import re
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler


def inspect_csv(df):
    print("===== CSV OVERVIEW =====")
    print(f"Rows    : {df.shape[0]}")
    print(f"Columns : {df.shape[1]}")
    print("\n--- Data types ---")
    print(df.dtypes)
    print("\n--- Memory usage ---")
    print(round(df.memory_usage(deep=True).sum() / 1024 ** 2, 2), "MB")


df_raw = pd.read_csv("../../data/merge/Khang_done.csv")
inspect_csv(df_raw)


===== CSV OVERVIEW =====
Rows    : 279
Columns : 12

--- Data types ---
Price_Billion    float64
Price_Per_m2     float64
Area_m2          float64
District          object
Ward              object
Bedrooms           int64
Is_MatTien         int64
Width_m          float64
Floors           float64
Is Hem             int64
Post Time         object
Toilets            int64
dtype: object

--- Memory usage ---
0.07 MB


In [22]:
def report_missing_values(df):
    print("===== MISSING VALUE REPORT =====")
    missing = df.isna().sum()
    percent = (missing / len(df)) * 100
    report = pd.DataFrame({
        "Missing_Count": missing,
        "Percent_Missing (%)": percent.round(2)
    }).sort_values(by="Missing_Count", ascending=False)
    print(report)
    return report


report_missing_values(df_raw)

===== MISSING VALUE REPORT =====
               Missing_Count  Percent_Missing (%)
Price_Billion              0                  0.0
Price_Per_m2               0                  0.0
Area_m2                    0                  0.0
District                   0                  0.0
Ward                       0                  0.0
Bedrooms                   0                  0.0
Is_MatTien                 0                  0.0
Width_m                    0                  0.0
Floors                     0                  0.0
Is Hem                     0                  0.0
Post Time                  0                  0.0
Toilets                    0                  0.0


,Missing_Count,Percent_Missing (%)
Price_Billion,0,0.0
Price_Per_m2,0,0.0
Area_m2,0,0.0
District,0,0.0
Ward,0,0.0
Bedrooms,0,0.0
Is_MatTien,0,0.0
Width_m,0,0.0
Floors,0,0.0
Is Hem,0,0.0
